<a href="https://colab.research.google.com/github/Asif1310/3004/blob/main/Convolutional_neural_network_for_face_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the required libraries

In [1]:
import numpy as np
import tensorflow as tf
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import zipfile
import os

# Importing the contents of github repository

In [2]:
!wget -O '/content/main.zip' 'https://github.com/Asif1310/3004/archive/refs/heads/main.zip'
zip = zipfile.ZipFile('/content/main.zip','r')
zip.extractall('/content')
zip.close()

--2022-03-01 08:27:19--  https://github.com/Asif1310/3004/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/Asif1310/3004/zip/refs/heads/main [following]
--2022-03-01 08:27:19--  https://codeload.github.com/Asif1310/3004/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.112.9
Connecting to codeload.github.com (codeload.github.com)|140.82.112.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/content/main.zip’

/content/main.zip       [   <=>              ]  30.58M  23.2MB/s    in 1.3s    

2022-03-01 08:27:20 (23.2 MB/s) - ‘/content/main.zip’ saved [32068422]



## Extracting the data folder 

In [3]:
zip = zipfile.ZipFile('/content/3004-main/data.zip','r')
zip.extractall('/content')
zip.close()

## Extracting the animated folder

In [4]:
zip = zipfile.ZipFile('/content/3004-main/animated.zip','r')
zip.extractall('/content')
zip.close()

# Preprocessing the images:

i) Training set

In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('/content/training',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 277 images belonging to 2 classes.


ii) Test set

In [6]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        '/content/test',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 30 images belonging to 2 classes.


# Building the model

In [7]:
model = tf.keras.models.Sequential() #creating an instance of Sequential class 
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3))) #first convolutional layer
model.add(tf.keras.layers.MaxPooling2D((2, 2))) #first sub-sampling layer
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu')) #second convolutional layer
model.add(tf.keras.layers.MaxPooling2D((2, 2))) #second sub-sampling layer
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu')) #third convolutional layer

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        36928     
                                                                 
Total params: 56,320
Trainable params: 56,320
Non-traina

Creating a flatten layer to convert the input into a vector

In [9]:
model.add(tf.keras.layers.Flatten())

Creating the fully connected layer and the final output layer 

In [10]:
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

#Compiling and training the model:

In [11]:
model.compile(optimizer='adam',
              loss= 'binary_crossentropy',
              metrics=['accuracy'])

In [12]:
model.fit(x = training_set, validation_data = test_set, epochs = 50)

Epoch 1/50
9/9 [==============================] - 8s 867ms/step - loss: 0.7651 - accuracy: 0.5054 - val_loss: 0.6926 - val_accuracy: 0.4333
Epoch 2/50
9/9 [==============================] - 4s 453ms/step - loss: 0.6931 - accuracy: 0.5126 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/50
9/9 [==============================] - 4s 446ms/step - loss: 0.6920 - accuracy: 0.5271 - val_loss: 0.6930 - val_accuracy: 0.5667
Epoch 4/50
9/9 [==============================] - 4s 437ms/step - loss: 0.6884 - accuracy: 0.5379 - val_loss: 0.6919 - val_accuracy: 0.5667
Epoch 5/50
9/9 [==============================] - 4s 440ms/step - loss: 0.6880 - accuracy: 0.6137 - val_loss: 0.6907 - val_accuracy: 0.6333
Epoch 6/50
9/9 [==============================] - 4s 453ms/step - loss: 0.6736 - accuracy: 0.6065 - val_loss: 0.6971 - val_accuracy: 0.5000
Epoch 7/50
9/9 [==============================] - 4s 457ms/step - loss: 0.6455 - accuracy: 0.6390 - val_loss: 0.8628 - val_accuracy: 0.4667
Epoch 8/50
9/9 [====

# Using the model to predict some images

Defining a function named 'load_predict' which upon passing the directory name as the parameter returns predictions on all the images stored by that directory

In [13]:
def load_predict(directory):
  prediction_names = []
  for i in os.listdir(directory):
    j = directory + i
    test1 = image.load_img(j, target_size = (64, 64))
    test1 = image.img_to_array(test1)
    test1 = np.expand_dims(test1, axis = 0)
    pred = model.predict(test1)
    if pred == 0:
      pred1 = "Ronaldo"
    else:
      pred1 = "messi"
    prediction_names.append(pred1)
  return prediction_names

i) Predictions on the test set of Ronaldo images

In [14]:
directory = '/content/test/class 0/'
pred = load_predict(directory)
print("Number of Ronaldo predictions : %2d" %(pred.count('Ronaldo')))
print("Number of Messi predictions : %2d" %(pred.count('messi')))

Number of Ronaldo predictions : 13
Number of Messi predictions :  2


ii) Predictions on the test set of Messi images

In [15]:
directory = '/content/test/class 1/'
pred = load_predict(directory)
print("Number of Ronaldo predictions : %2d" %(pred.count('Ronaldo')))
print("Number of Messi predictions : %2d" %(pred.count('messi')))

Number of Ronaldo predictions :  2
Number of Messi predictions : 13


iii) Predictions on the animated set of Ronaldo images

In [16]:
directory = '/content/Ronaldo_animated/'
pred = load_predict(directory)
print("Number of Ronaldo predictions : %2d" %(pred.count('Ronaldo')))
print("Number of Messi predictions : %2d" %(pred.count('messi')))

Number of Ronaldo predictions : 17
Number of Messi predictions :  8


iv) Predictions on the animated set of Messi images

In [17]:
directory = '/content/Messi_animated/'
pred = load_predict(directory)
print("Number of Ronaldo predictions : %2d" %(pred.count('Ronaldo')))
print("Number of Messi predictions : %2d" %(pred.count('messi')))

Number of Ronaldo predictions :  8
Number of Messi predictions : 17


Prediction on the famous 'bust' image of Ronaldo

In [18]:
zip = zipfile.ZipFile('/content/3004-main/Ronaldo Bust image.zip','r')
zip.extractall('/content')
zip.close()

In [19]:
test1 = image.load_img('/content/Ronaldo Bust image.jpg', target_size = (64, 64))
test1 = image.img_to_array(test1)
test1 = np.expand_dims(test1, axis = 0)
pred = model.predict(test1)
if pred==0:
  print('Ronaldo')
else:
  print('Messi')

Ronaldo
